<a href="https://colab.research.google.com/github/rmafonso/novaHdlAssist_AI/blob/main/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install Required Libraries
!pip install -q groq sentence-transformers PyPDF2 nltk chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.9/19.9 MB 67.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 44.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 67.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 4.9 MB/s eta 

In [ ]:
# Import Libraries and Set Up GROQ API
import os
from groq import Groq
from sentence_transformers import SentenceTransformer
import chromadb
import PyPDF2
from nltk.tokenize import word_tokenize
import nltk

# Download NLTK data for Portuguese tokenization
nltk.download('punkt_tab', quiet=True)

from google.colab import userdata

groq_client = Groq(
    api_key=userdata.get("groq_api_key2"),)

In [ ]:
!pip install --upgrade PyPDF2

In [ ]:
!pip install pikepdf

import pikepdf
import PyPDF2
import os

# Function to flatten PDF - REMOVED
# def flatten_pdf(input_path, output_path):
#   """Flattens the PDF using pikepdf."""
#   with pikepdf.open(input_path) as pdf:
#     pdf.save(output_path)

# Flatten the PDF before extraction - REMOVED
pdf_path = "/content/drive/MyDrive/Colab Notebooks/pdf db/VHDL_Modules.pdf"
# flattened_pdf_path = "/content/drive/MyDrive/Colab Notebooks/pdf db/flattened_VHDL_Modules.pdf"  # Update the flattened path as well - REMOVED
# flatten_pdf(pdf_path, flattened_pdf_path) # REMOVED

# ... (later in the code, when loading the PDF)
# document_paths = [flattened_pdf_path]  # Use the flattened path - REMOVED

# Load and Prepare PDF Documents (Mostrar 10 Primeiros Pedaços)
def extract_text_from_pdf(pdf_path):
  with open(pdf_path, 'rb') as file:
    reader = PyPDF2.PdfReader(file)  # Now use the original PDF
    text = ""
    for page in reader.pages:
      extracted = page.extract_text()
      if extracted:
        text += extracted + "\n"
    return text


# Load VHDL PDFs (use the original PDF path)
document_paths = [pdf_path] # Use the original path
documents = []
metadatas = []

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 32.0 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os # Import the os module
import PyPDF2 # Import PyPDF2
from nltk.tokenize import word_tokenize # Import word_tokenize

# Load and Prepare PDF Documents (Mostrar 10 Primeiros Pedaços)
def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page in reader.pages:
            extracted = page.extract_text()
            if extracted:
                text += extracted + "\n"
        return text

def split_into_chunks(text, chunk_size=400, overlap=100):
    words = word_tokenize(text, language='portuguese')
    chunks = []
    for i in range(0, len(words), chunk_size - overlap):
        chunk = " ".join(words[i:i + chunk_size])
        chunks.append(chunk)
    return chunks

# Directory containing the VHDL PDFs
pdf_directory = "/content/drive/MyDrive/Colab Notebooks/pdf db/" # Define the directory path

# Get all PDF file paths in the directory
document_paths = [os.path.join(pdf_directory, f) for f in os.listdir(pdf_directory) if f.endswith(".pdf")] # Get all PDF paths

documents = []
metadatas = []

for path in document_paths:
    text = extract_text_from_pdf(path)
    if not text.strip():
        print(f"Aviso: Nenhum texto extraído de {path}")
        continue
    chunks = split_into_chunks(text)
    documents.extend(chunks)
    metadatas.extend([{"source": path, "chunk_id": i} for i in range(len(chunks))])
    print(f"Processado {path}: {len(chunks)} pedaços")

print("Total de pedaços de documentos:", len(documents))

# Mostrar os 10 primeiros pedaços
print("\nOs 10 primeiros pedaços (ou menos, se houver menos):")
for i, (doc, meta) in enumerate(zip(documents[:10], metadatas[:10])):
    print(f"Pedaço {i} (de {meta['source']}, ID {meta['chunk_id']}):")
    print(f"Texto completo: {doc}")
    print(f"Tamanho: {len(doc.split())} palavras\n")

Processado /content/drive/MyDrive/Colab Notebooks/pdf db/VHDL_Modules.pdf: 5 pedaços
Total de pedaços de documentos: 5

Os 10 primeiros pedaços (ou menos, se houver menos):
Pedaço 0 (de /content/drive/MyDrive/Colab Notebooks/pdf db/VHDL_Modules.pdf, ID 0):
Texto completo: Bit Full Adder Description : library IEEE ; use IEEE.STD_LOGIC_1164.ALL ; entity < NAME > is port ( < INPUT_BIT1 > , < INPUT_BIT2 > , < INPUT_CARRY_BIT > : in std_logic ; < SUM_BIT > , < CARRY_BIT > : out std_logic ) ; end < NAME > ; architecture Behavioral of < NAME > is signal xor12 , carry12 : std_logic ; begin xor12 < = < INPUT_BIT1 > XOR < INPUT_BIT2 > ; carry12 < = < INPUT_BIT1 > AND < INPUT_BIT2 > ; < SUM_BIT > < = xor12 XOR c_in ; < CARRY_BIT > < = carry12 OR ( < INPUT_CARRY_BIT > AND xor12 ) ; end Behavioral ; Basic ALU Description : library IEEE ; use IEEE.STD_LOGIC_1164.ALL ; use IEEE.STD_LOGIC_signed.all ; entity < NAME > is -- The input numbers should be introduced in 2 's Complement form Port ( < OPERAND

In [ ]:
# Build Chroma Collection
embedder = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2')
client = chromadb.Client()  # In-memory client for Colab
collection_name = "vhdl_collection"

# Create or get collection
collection = client.get_or_create_collection(
    name=collection_name,
    metadata={"hnsw:space": "cosine"}
)

# If collection is empty, add documents
if collection.count() == 0:
    embeddings = embedder.encode(documents, convert_to_numpy=True, show_progress_bar=True).tolist()
    ids = [f"doc_{i}" for i in range(len(documents))]
    collection.add(
        documents=documents,
        embeddings=embeddings,
        metadatas=metadatas,
        ids=ids
    )
    print(f"Adicionados {len(documents)} pedaços à coleção Chroma")
else:
    print(f"Usando coleção existente com {collection.count()} pedaços")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Adicionados 5 pedaços à coleção Chroma


In [ ]:
# Define Retrieval Function with Chroma (Query Livre)
query = input("Digite sua pergunta (ou 'sair' para parar): ")
def retrieve_documents(query, embedder, collection, k=30):
    query_embedding = embedder.encode([query], convert_to_numpy=True).tolist()[0]
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=k,
        include=["documents", "metadatas"]
    )
    retrieved_docs = results["documents"][0]
    retrieved_metas = results["metadatas"][0]
    return retrieved_docs, retrieved_metas

# Test retrieval (deixe a query vazia para personalizar)
retrieved_docs, retrieved_metas = retrieve_documents(query, embedder, collection)
print("\nPedaços recuperados:")
for i, (doc, meta) in enumerate(zip(retrieved_docs, retrieved_metas), 1):
    print(f"Pedaço {i} (de {meta['source']}, ID {meta['chunk_id']}): {doc[:200]}...")


Pedaços recuperados:
Pedaço 1 (de /content/drive/MyDrive/Colab Notebooks/pdf db/VHDL_Modules.pdf, ID 2): a_aux ( < NUMBER_OF_BITS > ) = ' 1 ' and b_aux ( < NUMBER_OF_BITS > ) = ' 1 ' and result_aux ( < NUMBER_OF_BITS > ) = ' 0 ' ) else ' 0 ' ; subtract1_overflow < = ' 1 ' when ( a_aux ( < NUMBER_OF_BITS ...
Pedaço 2 (de /content/drive/MyDrive/Colab Notebooks/pdf db/VHDL_Modules.pdf, ID 1): ; begin -- A bit is added to both operands to save the result with carry included a_aux < = ' 0 ' & < OPERAND_A > when < OPERAND_A > ( < NUMBER_OF_BITS > ) = ' 0 ' else ' 1 ' ; b_aux < = ' 0 ' & < OPE...
Pedaço 3 (de /content/drive/MyDrive/Colab Notebooks/pdf db/VHDL_Modules.pdf, ID 4): -- when s_1 = > < NAME > _state < = s_2 ; end case ; end if ; end if ; end process ;...
Pedaço 4 (de /content/drive/MyDrive/Colab Notebooks/pdf db/VHDL_Modules.pdf, ID 0): Bit Full Adder Description : library IEEE ; use IEEE.STD_LOGIC_1164.ALL ; entity < NAME > is port ( < INPUT_BIT1 > , < INPUT_BIT2 > , < INPUT_CARR

In [ ]:
# Define Generation Function with GROQ
def generate_response(query, retrieved_docs, conversation_history, max_context_length=4000):
    context = "\n".join(str(doc) for doc in retrieved_docs)
    if len(context) > max_context_length:
        context = context[:max_context_length] + "... [truncado]"

    # Build prompt with context and conversation history
    prompt = (
        "Você é um assistente especialista em VHDL. Use apenas o contexto nos PDFs VDHL para responder com precisão em português. "
        "Se o contexto não tiver informações suficientes, diga isso e não diga mais nada.\n\n"
        "Histórico da Conversa:\n" + "\n".join(conversation_history) + "\n\n"
        "Contexto:\n" + context + "\n\n"
        "Pergunta: " + query
    )

    response = groq_client.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages=[
            {"role": "system", "content": "Você é um assistente especialista em VHDL."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=1500,
        temperature=0.7
    )
    return response.choices[0].message.content

# Test generation (this part will be removed in the interactive loop)
# answer = generate_response(query, retrieved_docs, []) # Removed for interactive loop
# print("\nResposta:", answer) # Removed for interactive loop

# Define Generation Function with GROQ
def generate_response(query, retrieved_docs, max_context_length=4000):
    #context = "\n".join(retrieved_docs)
    context = "\n".join(str(doc) for doc in retrieved_docs)
    if len(context) > max_context_length:
        context = context[:max_context_length] + "... [truncado]"

    prompt = (
        "Você é um assistente especialista em VHDL. Use apenas o contexto nos PDFs VDHL para responder com precisão em português. "
        "Se o contexto não tiver informações suficientes, diga isso e não diga mais nada.\n\n"
        "Contexto:\n" + context + "\n\n"
        "Pergunta: " + query
    )

    response = groq_client.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages=[
            {"role": "system", "content": "Você é um assistente especialista em VHDL."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=1500,
        temperature=0.7
    )
    return response.choices[0].message.content

# Test generation
answer = generate_response(query, retrieved_docs)
print("\nResposta:", answer)

In [ ]:
# Interactive Query Loop (Portuguese)
conversation_history = [] # Initialize conversation history
while True:
    query = input("Digite sua pergunta (ou 'sair' para parar): ")
    # print(conversation_history)
    if query.lower() == 'sair':
        break
    retrieved_docs, retrieved_metas = retrieve_documents(query, embedder, collection)
    print("\nPedaços recuperados:")
    for i, (doc, meta) in enumerate(zip(retrieved_docs, retrieved_metas), 1):
        print(f"Pedaço {i} (de {meta['source']}): {doc[:200]}...")

    # Generate response with conversation history
    answer = generate_response(query, retrieved_docs, conversation_history)
    print("\nResposta:", answer)

    # Update conversation history
    conversation_history.append(f"User: {query}")
    conversation_history.append(f"Assistant: {answer}")

Digite sua pergunta (ou 'sair' para parar): oq é um multiplexer

Pedaços recuperados:
Pedaço 1 (de /content/drive/MyDrive/Colab Notebooks/pdf db/VHDL_Modules.pdf): a_aux ( < NUMBER_OF_BITS > ) = ' 1 ' and b_aux ( < NUMBER_OF_BITS > ) = ' 1 ' and result_aux ( < NUMBER_OF_BITS > ) = ' 0 ' ) else ' 0 ' ; subtract1_overflow < = ' 1 ' when ( a_aux ( < NUMBER_OF_BITS ...
Pedaço 2 (de /content/drive/MyDrive/Colab Notebooks/pdf db/VHDL_Modules.pdf): Bit Full Adder Description : library IEEE ; use IEEE.STD_LOGIC_1164.ALL ; entity < NAME > is port ( < INPUT_BIT1 > , < INPUT_BIT2 > , < INPUT_CARRY_BIT > : in std_logic ; < SUM_BIT > , < CARRY_BIT > :...
Pedaço 3 (de /content/drive/MyDrive/Colab Notebooks/pdf db/VHDL_Modules.pdf): ; begin -- A bit is added to both operands to save the result with carry included a_aux < = ' 0 ' & < OPERAND_A > when < OPERAND_A > ( < NUMBER_OF_BITS > ) = ' 0 ' else ' 1 ' ; b_aux < = ' 0 ' & < OPE...
Pedaço 4 (de /content/drive/MyDrive/Colab Notebooks/pdf db/VHDL_Modu

In [4]:
# Passo 3: Inicializar Git no Colab

# Configurar o nome de utilizador do Git
!git config --global user.name "rmafonso"

# Configurar o email do utilizador do Git
!git config --global user.email "rm.afonso@campus.fct.unl.pt"

# Inicializar um repositório Git no diretório atual (onde o notebook está)
# Certifique-se de que está no diretório correto ou ajuste o caminho se necessário
!git init

print("Git inicializado e configurado no Colab.")

hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /content/.git/
Git inicializado e configurado no Colab.


In [8]:
# Passo 4: Adicionar e Commitar Ficheiros

# Criar um arquivo .gitignore para ignorar arquivos do Google Drive e outros
# Adicione outras linhas se houver outros arquivos/pastas que você queira ignorar
with open('.gitignore', 'w') as f:
    f.write("drive/\n")
    f.write(".config/\n")
    f.write("sample_data/\n")
    f.write("teste/\n")
    # Adicione aqui outros padrões de arquivos/pastas a ignorar, se necessário

print(".gitignore criado para ignorar a pasta 'drive', '.config' e 'sample_data' e 'teste'.")

# Adicionar o notebook atual explicitamente ao staging area
# O nome do notebook no Colab geralmente é "Nome_do_seu_notebook.ipynb"
# Você pode verificar o nome do seu notebook no título da página do Colab
# Substitua "Seu_Notebook_Colab.ipynb" pelo nome real do seu ficheiro .ipynb
# !git add "RAG.ipynb" # <--- Substitua pelo nome do seu ficheiro .ipynb

# Adicionar todos os outros ficheiros do diretório atual ao staging area (exceto os ignorados)
!git add .

# Criar o commit, incluindo o notebook e outros ficheiros
# A mensagem do commit deve ser descritiva
!git commit -m "Segundo commit: Adicionado o notebook RAG VHDL e atualizados ficheiros"

print("Ficheiros adicionados e commit criado com o notebook.")

.gitignore criado para ignorar a pasta 'drive', '.config' e 'sample_data' e 'teste'.
On branch master
nothing to commit, working tree clean
Ficheiros adicionados e commit criado com o notebook.


In [9]:
# Passo 5: Adicionar o Repositório Remoto

# Substitua o URL abaixo pelo URL do seu repositório no GitHub
remote_url = "https://github.com/rmafonso/novaHdlAssist_AI"

# Adicionar o repositório remoto
!git remote add origin {remote_url}

print(f"Repositório remoto adicionado: {remote_url}")

Repositório remoto adicionado: https://github.com/rmafonso/novaHdlAssist_AI


In [10]:
# Passo 6: Fazer Push das Alterações para o GitHub

# Importar userdata para aceder aos secrets
from google.colab import userdata
import os

# Obter o Personal Access Token (PAT) do GitHub do Secrets Manager
# Certifique-se de que guardou o seu PAT no Secrets Manager com o nome 'GITHUB_PAT'
github_pat = userdata.get('GITHUB_PAT')

# Obter o nome de utilizador do Git configurado anteriormente (ou defina-o se não o fez)
github_username = "rmafonso" # Substitua pelo seu nome de utilizador do GitHub

# Obter o URL remoto configurado anteriormente (do Passo 5)
# Certifique-se de que a variável remote_url do Passo 5 está definida e contém o URL correto.
# Se não tiver a certeza, pode obtê-lo novamente ou definir aqui:
remote_url = "https://github.com/rmafonso/novaHdlAssist_AI" # Substitua pelo seu URL

# Construir o URL com o token para autenticação HTTPS
# O formato é https://<username>:<token>@github.com/SeuNomeDeUtilizador/NomeDoSeuRepositorio.git
if remote_url.startswith("https://"):
    base_url = remote_url[len("https://"):]
    authenticated_url = f"https://{github_username}:{github_pat}@{base_url}"
else:
     # Se o URL não começar com https://, use-o como está e tente adicionar as credenciais
     authenticated_url = f"https://{github_username}:{github_pat}@{remote_url}"

# Agora execute o comando push usando o URL autenticado temporariamente
# git push <URL_autenticado> <branch>
# Substitua 'master' por 'main' se for o seu branch principal
# Certifique-se de que o branch local que está a empurrar é o correto (geralmente 'master' ou 'main')
!git push {authenticated_url} master

print("Alterações enviadas para o GitHub.")

To https://github.com/rmafonso/novaHdlAssist_AI
 ! [rejected]        master -> master (fetch first)
error: failed to push some refs to 'https://github.com/rmafonso/novaHdlAssist_AI'
hint: Updates were rejected because the remote contains work that you do
hint: not have locally. This is usually caused by another repository pushing
hint: to the same ref. You may want to first integrate the remote changes
hint: (e.g., 'git pull ...') before pushing again.
hint: See the 'Note about fast-forwards' in 'git push --help' for details.
Alterações enviadas para o GitHub.


In [13]:
# Certificar que estamos no diretório onde o notebook está guardado
# Substitua '/content/drive/MyDrive/Colab Notebooks/' pelo caminho correto para o seu notebook
notebook_dir = "/content/drive/MyDrive/Colab Notebooks/"
%cd {notebook_dir}

# Verificar o diretório atual
!pwd

# Re-inicializar o repositório Git (se já existir, git init simplesmente re-inicializa sem perda de dados)
!git init

# Adicionar o arquivo .gitignore novamente (garantir que está no diretório certo)
with open('.gitignore', 'w') as f:
    f.write("drive/\n")
    f.write(".config/\n")
    f.write("sample_data/\n")
    # Adicione aqui outros padrões de arquivos/pastas a ignorar, se necessário

print(".gitignore criado ou atualizado.")

# Adicionar o notebook atual explicitamente ao staging area
# Substitua "RAG-PDF_VHDL_Assistente_AI-Colab.ipynb" pelo nome real do seu ficheiro .ipynb
!git add "RAG-PDF_VHDL_Assistente_AI-Colab.ipynb"

# Adicionar todos os outros ficheiros relevantes no diretório atual ao staging area
# Por agora, vamos adicionar apenas o notebook e o .gitignore.
!git add .gitignore


# Criar o commit
# Vamos verificar o status antes de commitar para ver se há algo a ser commitado
!git status
!git commit -m "Segundo commit: Adicionado o notebook RAG VHDL e .gitignore"

print("Ficheiros adicionados e commit criado (se houver alterações).")

/content/drive/MyDrive/Colab Notebooks
/content/drive/MyDrive/Colab Notebooks
hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /content/drive/MyDrive/Colab Notebooks/.git/
.gitignore criado ou atualizado.
fatal: pathspec 'RAG-PDF_VHDL_Assistente_AI-Colab.ipynb' did not match any files
On branch master

No commits yet

Changes to be committed:
  (use "git rm --cached <file>..." to unstage)
	new file:   .gitignore

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	RAG 12 7
	RAG 13 6
	RAG.ipyn

In [18]:
# Passo 10: Puxar Alterações do Repositório Remoto Novamente (Pull)

# Certificar que estamos no diretório correto (onde o repositório foi inicializado)
notebook_dir = "/content/drive/MyDrive/Colab Notebooks/"
%cd {notebook_dir}

# Importar userdata para aceder aos secrets (se necessário para autenticação)
from google.colab import userdata
import os

# Obter o Personal Access Token (PAT) do GitHub do Secrets Manager
github_pat = userdata.get('GITHUB_PAT')

# Obter o nome de utilizador do Git
github_username = "rmafonso" # Substitua pelo seu nome de utilizador do GitHub

# Obter o URL remoto
remote_url = "https://github.com/rmafonso/novaHdlAssist_AI" # Substitua pelo seu URL

# Construir o URL autenticado para o pull
if remote_url.startswith("https://"):
    base_url = remote_url[len("https://"):]
    authenticated_url = f"https://{github_username}:{github_pat}@{base_url}"
else:
     authenticated_url = f"https://{github_username}:{github_pat}@{remote_url}"

# Configurar a estratégia de pull para rebase para esta branch
!git config branch.master.pullRebase true

# Executar o comando pull
# Com a configuração acima, um pull normal tentará rebasear
!git pull {authenticated_url} master

print("Alterações puxadas do GitHub novamente (configurado para rebase).")

/content/drive/MyDrive/Colab Notebooks
From https://github.com/rmafonso/novaHdlAssist_AI
 * branch            master     -> FETCH_HEAD
hint: You have divergent branches and need to specify how to reconcile them.
hint: You can do so by running one of the following commands sometime before
hint: your next pull:
hint: 
hint:   git config pull.rebase false  # merge (the default strategy)
hint:   git config pull.rebase true   # rebase
hint:   git config pull.ff only       # fast-forward only
hint: 
hint: You can replace "git config" with "git config --global" to set a default
hint: preference for all repositories. You can also pass --rebase, --no-rebase,
hint: or --ff-only on the command line to override the configured default per
hint: invocation.
fatal: Need to specify how to reconcile divergent branches.
Alterações puxadas do GitHub novamente.


In [19]:
# Passo 11: Fazer Push das Alterações para o GitHub Novamente

# Certificar que estamos no diretório correto
notebook_dir = "/content/drive/MyDrive/Colab Notebooks/"
%cd {notebook_dir}

# Importar userdata para aceder aos secrets
from google.colab import userdata
import os

# Obter o Personal Access Token (PAT) do GitHub do Secrets Manager
github_pat = userdata.get('GITHUB_PAT')

# Obter o nome de utilizador do Git
github_username = "rmafonso" # Substitua pelo seu nome de utilizador do GitHub

# Obter o URL remoto
remote_url = "https://github.com/rmafonso/novaHdlAssist_AI" # Substitua pelo seu URL

# Construir o URL com o token para autenticação HTTPS
if remote_url.startswith("https://"):
    base_url = remote_url[len("https://"):]
    authenticated_url = f"https://{github_username}:{github_pat}@{base_url}"
else:
     authenticated_url = f"https://{github_username}:{github_pat}@{remote_url}"

# Agora execute o comando push
# git push <URL_autenticado> <branch>
!git push {authenticated_url} master

print("Alterações enviadas para o GitHub (segunda tentativa após pull).")

/content/drive/MyDrive/Colab Notebooks
To https://github.com/rmafonso/novaHdlAssist_AI
 ! [rejected]        master -> master (non-fast-forward)
error: failed to push some refs to 'https://github.com/rmafonso/novaHdlAssist_AI'
hint: Updates were rejected because the tip of your current branch is behind
hint: its remote counterpart. Integrate the remote changes (e.g.
hint: 'git pull ...') before pushing again.
hint: See the 'Note about fast-forwards' in 'git push --help' for details.
Alterações enviadas para o GitHub (segunda tentativa após pull).


In [17]:
# Passo 9: Fazer Push das Alterações para o GitHub (Após o Pull)

# Certificar que estamos no diretório correto
notebook_dir = "/content/drive/MyDrive/Colab Notebooks/"
%cd {notebook_dir}

# Importar userdata para aceder aos secrets
from google.colab import userdata
import os

# Obter o Personal Access Token (PAT) do GitHub do Secrets Manager
github_pat = userdata.get('GITHUB_PAT')

# Obter o nome de utilizador do Git
github_username = "rmafonso" # Substitua pelo seu nome de utilizador do GitHub

# Obter o URL remoto
remote_url = "https://github.com/rmafonso/novaHdlAssist_AI" # Substitua pelo seu URL

# Construir o URL com o token para autenticação HTTPS
if remote_url.startswith("https://"):
    base_url = remote_url[len("https://"):]
    authenticated_url = f"https://{github_username}:{github_pat}@{base_url}"
else:
     authenticated_url = f"https://{github_username}:{github_pat}@{remote_url}"

# Agora execute o comando push
# git push <URL_autenticado> <branch>
# Substitua 'master' por 'main' se for o seu branch principal
!git push {authenticated_url} master

print("Alterações enviadas para o GitHub.")

/content/drive/MyDrive/Colab Notebooks
To https://github.com/rmafonso/novaHdlAssist_AI
 ! [rejected]        master -> master (non-fast-forward)
error: failed to push some refs to 'https://github.com/rmafonso/novaHdlAssist_AI'
hint: Updates were rejected because the tip of your current branch is behind
hint: its remote counterpart. Integrate the remote changes (e.g.
hint: 'git pull ...') before pushing again.
hint: See the 'Note about fast-forwards' in 'git push --help' for details.
Alterações enviadas para o GitHub.


In [16]:
# Passo 8: Puxar Alterações do Repositório Remoto (Pull)

# Certificar que estamos no diretório correto (onde o repositório foi inicializado)
# Substitua '/content/drive/MyDrive/Colab Notebooks/' pelo caminho correto
notebook_dir = "/content/drive/MyDrive/Colab Notebooks/"
%cd {notebook_dir}

# Importar userdata para aceder aos secrets (se necessário para autenticação)
from google.colab import userdata
import os

# Obter o Personal Access Token (PAT) do GitHub do Secrets Manager
github_pat = userdata.get('GITHUB_PAT')

# Obter o nome de utilizador do Git
github_username = "rmafonso" # Substitua pelo seu nome de utilizador do GitHub

# Obter o URL remoto
remote_url = "https://github.com/rmafonso/novaHdlAssist_AI" # Substitua pelo seu URL

# Construir o URL autenticado para o pull (geralmente não é necessário para pull de repositórios públicos, mas útil para privados ou se o remote_url não estiver configurado corretamente)
if remote_url.startswith("https://"):
    base_url = remote_url[len("https://"):]
    authenticated_url = f"https://{github_username}:{github_pat}@{base_url}"
else:
     authenticated_url = f"https://{github_username}:{github_pat}@{remote_url}"


# Executar o comando pull permitindo históricos não relacionados
# git pull <URL_autenticado> <branch> --allow-unrelated-histories
# Substitua 'master' por 'main' se for o seu branch principal
!git pull {authenticated_url} master --allow-unrelated-histories


print("Alterações puxadas do GitHub (permitindo históricos não relacionados).")

/content/drive/MyDrive/Colab Notebooks
From https://github.com/rmafonso/novaHdlAssist_AI
 * branch            master     -> FETCH_HEAD
hint: You have divergent branches and need to specify how to reconcile them.
hint: You can do so by running one of the following commands sometime before
hint: your next pull:
hint: 
hint:   git config pull.rebase false  # merge (the default strategy)
hint:   git config pull.rebase true   # rebase
hint:   git config pull.ff only       # fast-forward only
hint: 
hint: You can replace "git config" with "git config --global" to set a default
hint: preference for all repositories. You can also pass --rebase, --no-rebase,
hint: or --ff-only on the command line to override the configured default per
hint: invocation.
fatal: Need to specify how to reconcile divergent branches.
Alterações puxadas do GitHub (permitindo históricos não relacionados).


In [14]:
# Passo 7: Fazer Push das Alterações para o GitHub

# Importar userdata para aceder aos secrets
from google.colab import userdata
import os

# Obter o Personal Access Token (PAT) do GitHub do Secrets Manager
# Certifique-se de que guardou o seu PAT no Secrets Manager com o nome 'GITHUB_PAT'
github_pat = userdata.get('GITHUB_PAT')

# Obter o nome de utilizador do Git configurado anteriormente (ou defina-o se não o fez)
github_username = "rmafonso" # Substitua pelo seu nome de utilizador do GitHub

# Obter o URL remoto configurado anteriormente (do Passo 5)
# Certifique-se de que a variável remote_url do Passo 5 está definida e contém o URL correto.
# Se não tiver a certeza, pode obtê-lo novamente ou definir aqui:
remote_url = "https://github.com/rmafonso/novaHdlAssist_AI" # Substitua pelo seu URL

# Construir o URL com o token para autenticação HTTPS
# O formato é https://<username>:<token>@github.com/SeuNomeDeUtilizador/NomeDoSeuRepositorio.git
if remote_url.startswith("https://"):
    base_url = remote_url[len("https://"):]
    authenticated_url = f"https://{github_username}:{github_pat}@{base_url}"
else:
     # Se o URL não começar com https://, use-o como está e tente adicionar as credenciais
     authenticated_url = f"https://{github_username}:{github_pat}@{remote_url}"

# Agora execute o comando push usando o URL autenticado temporariamente
# git push <URL_autenticado> <branch>
# Substitua 'master' por 'main' se for o seu branch principal
# Certifique-se de que o branch local que está a empurrar é o correto (geralmente 'master' ou 'main')
!git push {authenticated_url} master

print("Alterações enviadas para o GitHub.")

To https://github.com/rmafonso/novaHdlAssist_AI
 ! [rejected]        master -> master (fetch first)
error: failed to push some refs to 'https://github.com/rmafonso/novaHdlAssist_AI'
hint: Updates were rejected because the remote contains work that you do
hint: not have locally. This is usually caused by another repository pushing
hint: to the same ref. You may want to first integrate the remote changes
hint: (e.g., 'git pull ...') before pushing again.
hint: See the 'Note about fast-forwards' in 'git push --help' for details.
Alterações enviadas para o GitHub.


# Interactive Query Loop (Portuguese)
while True:
    query = input("Digite sua pergunta (ou 'sair' para parar): ")
    if query.lower() == 'sair':
        break
    retrieved_docs, retrieved_metas = retrieve_documents(query, embedder, collection)
    print("\nPedaços recuperados:")
    for i, (doc, meta) in enumerate(zip(retrieved_docs, retrieved_metas), 1):
        print(f"Pedaço {i} (de {meta['source']}): {doc[:200]}...")
    answer = generate_response(query, retrieved_docs)
    print("\nResposta:", answer)

In [ ]:
# Listar os 10 Primeiros Pedaços do Chroma
def list_first_10_chunks(collection):
    total_chunks = collection.count()
    print(f"Total de pedaços na coleção: {total_chunks}")

    # Recuperar os 10 primeiros pedaços
    first_10_ids = [f"doc_{i}" for i in range(min(10, total_chunks))]
    results = collection.get(
        ids=first_10_ids,
        include=["documents", "metadatas"]
    )

    print("\nOs 10 primeiros pedaços no Chroma:")
    for i, (doc, meta) in enumerate(zip(results["documents"], results["metadatas"])):
        print(f"Pedaço {i} (de {meta['source']}, ID {meta['chunk_id']}):")
        print(f"Texto completo: {doc}")
        print(f"Tamanho: {len(doc.split())} palavras\n")

# Executar a listagem
list_first_10_chunks(collection)

Total de pedaços na coleção: 5

Os 10 primeiros pedaços no Chroma:
Pedaço 0 (de /content/drive/MyDrive/Colab Notebooks/pdf db/VHDL_Modules.pdf, ID 0):
Texto completo: Bit Full Adder Description : library IEEE ; use IEEE.STD_LOGIC_1164.ALL ; entity < NAME > is port ( < INPUT_BIT1 > , < INPUT_BIT2 > , < INPUT_CARRY_BIT > : in std_logic ; < SUM_BIT > , < CARRY_BIT > : out std_logic ) ; end < NAME > ; architecture Behavioral of < NAME > is signal xor12 , carry12 : std_logic ; begin xor12 < = < INPUT_BIT1 > XOR < INPUT_BIT2 > ; carry12 < = < INPUT_BIT1 > AND < INPUT_BIT2 > ; < SUM_BIT > < = xor12 XOR c_in ; < CARRY_BIT > < = carry12 OR ( < INPUT_CARRY_BIT > AND xor12 ) ; end Behavioral ; Basic ALU Description : library IEEE ; use IEEE.STD_LOGIC_1164.ALL ; use IEEE.STD_LOGIC_signed.all ; entity < NAME > is -- The input numbers should be introduced in 2 's Complement form Port ( < OPERAND_A > : in STD_LOGIC_VECTOR ( < NUMBER_OF_BITS > downto 0 ) ; < OPERAND_B > : in STD_LOGIC_VECTOR ( < NUMBE